<a href="https://colab.research.google.com/github/hyaesung3217/Cooling-Tower-Calculation/blob/main/CoolingTowerOptimization_ForUser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The model should be created and loaded (`polynomial_regression_model.pkl`) along with the polynomial transformer file (`polynomial_features.pkl`).  

These files must be placed directly under the `sample_data` folder, which can be accessed via the file icon located on the left.



#Checking GPU and CPU Functionality

By default, Google Colab runs on a CPU environment. However, in this program, it is configured to use a T4 GPU. If connecting to a GPU is not possible, it will automatically switch to a CPU environment.

To verify whether you are using a GPU or a CPU, simply check the following section. If "CUDA" appears, it indicates that the execution environment is a GPU. Otherwise, the execution environment is a CPU.

Although it was not implemented here due to the limitation of having only one GPU, having more GPUs significantly reduces execution time, particularly for sections 3 and 3-1. In such cases, parallelization would be the method applied.


In [ ]:
#Check GPU availability
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU working")
else:
    print("CUDA is not available. CPU is working")

CUDA is not available. CPU is working



# 0. Polynomial Equation


With the model generated from the file "Cooling Tower Parametrization," it is possible to extract and export a matrix of coefficients and related variables.

A message is printed: "Equation exported as 'polynomial_equation.xlsx'," and it is located in the left column.

In [ ]:

#Export cubic equation in excel

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
# El modelo se encuentra en:

warnings.filterwarnings('ignore')

#Open model
with open('polynomial_regression_model.pkl', 'rb') as file:
    model_poly = pickle.load(file)

#Open polynomial features
with open('polynomial_features.pkl', 'rb') as file:
    poly = pickle.load(file)

# Accessing model parameters
coefficients = model_poly.coef_
intercept = model_poly.intercept_
polynomial_degree = poly.degree

variable_names = {
    1: "Water flow",
    2: "Fan speed",
    3: "Range",
    4: "Wet bulb"
}

equation_parts = []
equation_parts.append(f"{intercept}")

for i, coef in enumerate(coefficients):
  if coef != 0:
    variable_index = i+1
    if variable_index in variable_names:
      equation_parts.append(f"{coef} * {variable_names[variable_index]}")
    else:
      equation_parts.append(f"{coef} * x_{variable_index}")


# Create a DataFrame for Excel export
data = {'Term': ['Intercept'] + [f'x_{i+1}' if i+1 not in variable_names else variable_names[i+1] for i in range(len(coefficients))],
        'Coefficient': [intercept] + list(coefficients)}
df = pd.DataFrame(data)

# Export to Excel
df.to_excel('polynomial_equation.xlsx', index=False)

print("Equation exported as 'polynomial_equation.xlsx'")



Equation exported as 'polynomial_equation.xlsx'


# Ayuda para el usuario

The following inputs, constants, and functions are often repeated across different programs. It is possible to modify their content. However, if any of them are changed, consider how far their change would affect (check places where the same name is used).

User-defined inputs can be entered manually or via the slider. The inputs are:

- **water flow**  
- **Wet bulb temperature**  
- **Número de stream**  
- **Minimum fan speed percentage**  
- **Initial range value (iteration)**  

## User-defined constant values (values can be changed)

- **CHILLER_CAPACITY_UNIT_TR**  
- **SPECIFIC_HEAT**  
- **CHILLER_WATER_FLOW**  
- **FAN_POWER**  

- **WATER_DENSITY**  
- **GRAVITY**  
- **PUMP_HEAD**  
- **PUMP_WATER_FLOW**  
- **PUMP_EFFECTIVENESS**  

## User-defined functions

- **chiller_consumption_calculation** --> To calculate chiller consumption  
- **new_range_calculation** --> To calculate a new range value after ONE iteration  
- **pump_calculation** --> To calculate pump consumption  
- **effectivenes_calculation** --> To calculate efficiency  
- **total_consumption_calculation** --> To calculate total consumption  
- **fan_consumption_calculation** --> To calculate fan consumption  
- **heat_rejected_by_condenser_calculation** --> To calculate heat rejected by the chiller condenser  
- **new_reange_calculation** --> Encapsulation of all the above functions, with the output of effectiveness, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption  

# Execute before using any section (Define constants and functions, only once)

In case of a function error (function not found/defined), execute again.







In [ ]:

#Constants

CHILLER_CAPACITY_UNIT_TR = ## Should be defined by user
SPECIFIC_HEAT = 4.18
CHILLER_WATER_FLOW = 8## Should be defined by user #kg/s
FAN_POWER = ## Should be defined by user #(kW)
WATER_DENSITY = 1000 #kg/m3
GRAVITY = 9.81 #m/s2
PUMP_HEAD = ## Should be defined by user #m
PUMP_WATER_FLOW = ## Should be defined by user #m3/s
PUMP_EFFECTIVENESS = 0.9 #rendimiento

PUMP_CALCULATION_COEFFICIENT = ##Defined by user
PUMP_CALCULATION_BIAS = ##Defined by user

#Function fan number
def calculate_fan_number(n_stream):
  if n_stream == 4:
    n_fan = 6
  else:
    n_fan = n_stream * 2
  return n_fan

#Function - small tools

def chiller_consumption_calculation(effectivenes: float,n_stream: float) -> float:
  chiller_consumption = effectivenes * CHILLER_CAPACITY_UNIT_TR * n_stream
  return chiller_consumption

def new_range_calculation(n_stream: float,heat_rejected_by_condenser: float,water_flow: float) -> float:
  new_range = heat_rejected_by_condenser / (n_stream * SPECIFIC_HEAT * CHILLER_WATER_FLOW*(water_flow/100)) #
  return new_range

def pump_calculation(n_stream: float, water_flow: float) -> float:
  pump_power = (WATER_DENSITY*GRAVITY*PUMP_HEAD*PUMP_WATER_FLOW*((water_flow/100)**2.7))/(PUMP_EFFECTIVENESS*1000)
  pump_consumption = pump_power*n_stream
  return pump_consumption

def effectivenes_calculation(cold_water_temperature: float,range: float) -> float:
  effectivenes = PUMP_CALCULATION_COEFFICIENT*(cold_water_temperature+range) + PUMP_CALCULATION_BIAS
  return effectivenes

def total_consumption_calculation(fan_consumption: float, chiller_consumption: float,n_stream: float, pump_consumption: float) -> float:
  total_chiller_consumption = fan_consumption + chiller_consumption + pump_consumption
  return total_chiller_consumption

def fan_consumption_calculation(n_fan: float, fan_speed: float) -> float:
  fan_consumption = fan_speed**2.7*FAN_POWER*n_fan
  return fan_consumption

def heat_rejected_by_condenser_calculation(chiller_consumption: float,n_stream: float) -> float:
  heat_rejected_by_condenser = chiller_consumption + (CHILLER_CAPACITY_UNIT_TR * n_stream * 3.51685)
  return heat_rejected_by_condenser

#Pipeline
def new_reange_calculation(predicted_y: float,n_stream: float,n_fan: float,fan_speed: float,water_flow: float,range: float) -> float:
  effectivenes = effectivenes_calculation(predicted_y,range)
  chiller_consumption = chiller_consumption_calculation(effectivenes, n_stream)
  heat_rejected_by_condenser = heat_rejected_by_condenser_calculation(chiller_consumption, n_stream)
  fan_consumption = fan_consumption_calculation(n_fan, fan_speed / 100)  # Normalize fan_speed
  pump_consumption = pump_calculation(n_stream,water_flow)
  total_consumption = total_consumption_calculation(fan_consumption, chiller_consumption,n_stream,pump_consumption)
  new_range = new_range_calculation(n_stream, heat_rejected_by_condenser,water_flow)
  return effectivenes, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption


# 1. Fan Speed vs Total Consumption
Based on the user-defined inputs (water flow, wet bulb temperature, n_stream, minimum fan speed), the user can generate a plot displaying the consumption graphs at various fan speeds (chiller consumption, pump consumption, fan consumption, total consumption).

The goal of this section is to identify the optimal point of minimum consumption.

In [ ]:
#1. Plot Fan speed vs Total consumption (2D)


import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display


warnings.filterwarnings('ignore')

#Open model
with open('polynomial_regression_model.pkl', 'rb') as file:
    model_poly = pickle.load(file)

#Open polynomial features
with open('polynomial_features.pkl', 'rb') as file:
    poly = pickle.load(file)

#Function fan number
def calculate_fan_number(n_stream):
  if n_stream == 4:
    n_fan = 6
  else:
    n_fan = n_stream * 2
  return n_fan


# Variable slidar configuration
water_flow_slider = widgets.FloatSlider(value=67, min=50, max=100, step=1, description='Water Flow(%):',style={'description_width': '100px'})
x_val_slider = widgets.FloatSlider(value=18.1, min=0, max=100, step=0.1, description='Inlet Wet Bulb Temp:',style={'description_width': '120px'})
n_stream_slider = widgets.FloatSlider(value=4, min=1, max=4, step=1, description='Number of Streams:',style={'description_width': '120px'})
minimum_fan_speed_percentage = float(input("Minimum percentage of fan speed from 100%: "))
initial_range_val = float(input("Initial range value: ")) #initial range

#Limit to range variation
threshold = 0.001


def update_plot(water_flow, x_val, n_stream):

    global results_min_df
    results = []
    results_min = []
    fan_speeds = np.arange(100, minimum_fan_speed_percentage-1, -1)

    #fan number calculation
    n_fan = calculate_fan_number(n_stream)


    for fan_speed in fan_speeds:


        #Cold water tempearture prediction
        new_input = np.array([[water_flow, fan_speed, initial_range_val, x_val]])
        new_input_poly = poly.transform(new_input)
        predicted_y = model_poly.predict(new_input_poly)[0]

        #Calculation
        effectivenes, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption = new_reange_calculation(predicted_y, n_stream, n_fan, fan_speed, water_flow,initial_range_val )


        previous_range = initial_range_val

        #Iterative loop
        while abs(new_range - previous_range) > threshold:

          previous_range = new_range

          #Cold water tempearture prediction
          new_input = np.array([[water_flow, fan_speed, previous_range, x_val]])
          new_input_poly = poly.transform(new_input)
          predicted_y = model_poly.predict(new_input_poly)[0]

          #Calculation
          effectivenes, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption = new_reange_calculation(predicted_y, n_stream, n_fan, fan_speed, water_flow,initial_range_val )


        results.append([fan_speed,new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption, predicted_y])

    #Get results_df
    print(f"Number of stream: {n_stream}")
    print(f"Number of fan: {n_fan}")
    print(f"Water flow : {water_flow}%")

    results_df = pd.DataFrame(results, columns=['Fan Speed (%)','New range', 'Total Consumption (kWe)', 'Fan Consumption (kW)', 'Chiller Consumption (kWe)', 'Condenser Pumps Consumption (kWe)','Cold water temperature (ºC)'])

    #Plot Total consumption, Fan Speed
    plt.figure(figsize=(12, 6))
    plt.plot(results_df['Fan Speed (%)'], results_df['Total Consumption (kWe)'], label='Total Consumption')
    plt.plot(results_df['Fan Speed (%)'], results_df['Fan Consumption (kW)'], label='Fan Consumption')
    plt.plot(results_df['Fan Speed (%)'], results_df['Chiller Consumption (kWe)'], label='Chiller Consumption')
    plt.plot(results_df['Fan Speed (%)'], results_df['Condenser Pumps Consumption (kWe)'], label='Pump Consumption')

    min_consumption_index = results_df['Total Consumption (kWe)'].idxmin()

    plt.scatter(results_df['Fan Speed (%)'][min_consumption_index], results_df['Total Consumption (kWe)'][min_consumption_index], color='red', label=f'Minimum Consumption: {results_df["Total Consumption (kWe)"][min_consumption_index]:.2f} kWe in {results_df["Fan Speed (%)"][min_consumption_index]:.2f}%')

    plt.xlabel('Fan Speed (%)')
    plt.ylabel('Total Consumption and Chiller Consumption (kWe)')
    plt.title('Consumption vs. Fan Speed')
    plt.legend()
    plt.grid(True)
    plt.show()


    return results_df

interact(update_plot, water_flow=water_flow_slider, x_val=x_val_slider, n_stream=n_stream_slider);



Minimum percentage of fan speed from 100%: 50
Initial range value: 50


interactive(children=(FloatSlider(value=67.0, description='Water Flow(%):', min=50.0, step=1.0, style=SliderSt…

# 2. Fan Speed vs Total Consumption vs Flow Rate (3D)

Using the user-defined inputs (wet bulb temperature, number of streams, minimum water flow percentage, minimum fan speed percentage, initial range value), the user can generate a 3D plot indicating the optimal point (minimum consumption) based on various flow rate and fan speed values.  

The goal of this section is to observe the variation in total consumption across different flow rate and fan speed values and identify the point of minimum consumption.

In [ ]:
#2. Plot Fan speed - Total consumption - Flow rate (3D)

#1. Plot Fan speed vs Total consumption


import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

# El modelo se encuentra en:

warnings.filterwarnings('ignore')

#Open model
with open('polynomial_regression_model.pkl', 'rb') as file:
    model_poly = pickle.load(file)

#Open polynomial features
with open('polynomial_features.pkl', 'rb') as file:
    poly = pickle.load(file)


#Function fan number
def calculate_fan_number(n_stream):
  if n_stream == 4:
    n_fan = 6
  else:
    n_fan = n_stream * 2
  return n_fan


# Variable slidar configuration
x_val_slider = widgets.FloatSlider(value=20, min=0, max=100, step=0.1, description='Inlet Wet Bulb Temp:',style={'description_width': '120px'})
n_stream_slider = widgets.FloatSlider(value=4, min=1, max=4, step=1, description='Number of Streams:',style={'description_width': '120px'})
minimum_water_flow_percentage = float(input("Minimum percentage of water flow from 100%: "))
minimum_fan_speed_percentage = float(input("Minimum percentage of fan speed from 100%: "))
initial_range_val = float(input("Initial range value: ")) #initial range

#Limit to range variation
threshold = 0.001

def update_plot(x_val, n_stream):
    global results_min_df
    results = []
    fan_speeds = np.arange(100, minimum_fan_speed_percentage-1, -1)
    water_flows = np.arange(100, minimum_water_flow_percentage-1, -1)

    X, Y = np.meshgrid(fan_speeds, water_flows)
    Z = np.zeros_like(X)

    # numero de fan
    n_fan = calculate_fan_number(n_stream)

    for i, fan_speed in enumerate(fan_speeds):
      for j, water_flow in enumerate(water_flows):

        #Cold water tempearture prediction
        new_input = np.array([[water_flow, fan_speed, initial_range_val, x_val]])
        new_input_poly = poly.transform(new_input)
        predicted_y = model_poly.predict(new_input_poly)[0]

        # Calculation
        effectivenes, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption = new_reange_calculation(predicted_y, n_stream, n_fan, fan_speed, water_flow,initial_range_val )

        previous_range = initial_range_val

        previous_chiller_consumption = 12000

        # Iteration for range calculation
        while abs(new_range - previous_range) > threshold and abs(previous_chiller_consumption - chiller_consumption) > threshold:
            previous_chiller_consumption = chiller_consumption
            previous_range = new_range
            new_input = np.array([[water_flow, fan_speed, previous_range, x_val]])
            new_input_poly = poly.transform(new_input)
            predicted_y = model_poly.predict(new_input_poly)[0]
            effectivenes, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption = new_reange_calculation(predicted_y, n_stream, n_fan, fan_speed, water_flow,previous_range)

        #Asignar el valor a Z
        Z[i, j] = total_consumption
        #print(Z)

        #Sacar consumo con fan speed y water flow
        results.append([fan_speed, water_flow, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption, predicted_y, effectivenes, new_range+predicted_y])

    #Por legibilidad, se guarda el dataframe de resultados en 2D.
    results_df = pd.DataFrame(results, columns=['Fan Speed (%)','Water flow (%)','New range', 'Total Consumption (kWe)', 'Fan Consumption (kW)', 'Chiller Consumption (kWe)', 'Condenser Pumps Consumption (kWe)','Cold water temperature (ºC)', 'effeciency (kW/RT)', 'CWLT (ºC)'])



    # Plotting the 3D surface graph
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.5)

    # Find the minimum total consumption
    min_consumption_index = results_df['Total Consumption (kWe)'].idxmin()
    min_fan_speed = results_df['Fan Speed (%)'][min_consumption_index]
    min_water_flow = results_df['Water flow (%)'][min_consumption_index]
    min_consumption = results_df['Total Consumption (kWe)'][min_consumption_index]

    max_consumpiton_index = results_df['Total Consumption (kWe)'].idxmax()
    max_fan_speed = results_df['Fan Speed (%)'][max_consumpiton_index]
    max_water_flow = results_df['Water flow (%)'][max_consumpiton_index]
    max_consumption = results_df['Total Consumption (kWe)'][max_consumpiton_index]

    # Highlight the minimum and value on the plot
    ax.scatter(min_water_flow, min_fan_speed, min_consumption, color='red', s=80, label=f'Minimum Consumption: {min_consumption:.2f} kWe\nFan Speed: {min_fan_speed:.2f}%, Water Flow: {min_water_flow:.2f}% with {n_stream} streams')
    ax.scatter(max_water_flow, max_fan_speed, max_consumption, color='blue', s=80, label=f'Maximum Consumption: {max_consumption:.2f} kWe\nFan Speed: {max_fan_speed:.2f}%, Water Flow: {max_water_flow:.2f}% with {n_stream} streams')

    # Highlight the minimum value on the plot
    #ax.scatter(min_water_flow, min_fan_speed, Z[min_idx], color='r', s=30, label='Minimum')

    ax.set_xlabel('Water Flow (%)')
    ax.set_ylabel('Fan Speed (%)')
    ax.set_zlabel('Total Consumption (kWe)')
    ax.set_title(f'Total Consumption Graph up {water_flow}% water flow, {fan_speed}% fan speed and {x_val}(ºC) wet bulb')
    ax.legend()

    plt.show()


    # Export to Excel
    # results_df.to_excel('minimum_consumption.xlsx', index=False)

    # print("Equation exported as 'minimum_consumption.xlsx'")


    return results_df



interact(update_plot, x_val=x_val_slider, n_stream=n_stream_slider);



FileNotFoundError: [Errno 2] No such file or directory: 'polynomial_regression_model.pkl'

# 3. Fan Speed vs Wet Bulb Temperature

Using the user-defined inputs (water flow, number of streams, initial range value (default set to 8)), the user can generate a 2D plot showing the fan speed values at different wet bulb temperature values when consumption is minimized.  

The goal of this section is to identify the fan speed points corresponding to minimum consumption across various wet bulb temperature levels.

In [ ]:
#3. Plot Inlet Wet bulb temperature vs Fan speed in differnet number of streams + PUMP power APPLIED

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import time
from mpl_toolkits.mplot3d import axes3d

#ejecucin paralela
from joblib import Parallel, delayed

# Medir tiempo

start_time = time.time()

warnings.filterwarnings('ignore')

#Open model
with open('polynomial_regression_model.pkl', 'rb') as file:
    model_poly = pickle.load(file)

#Open polynomial features
with open('polynomial_features.pkl', 'rb') as file:
    poly = pickle.load(file)

# Variable slidar configuration
water_flow_slider = widgets.FloatSlider(value=67, min=50, max=100, step=1, description='Water Flow(%):',style={'description_width': '100px'})
n_stream_slider = widgets.FloatSlider(value=4, min=1, max=4, step=1, description='Number of Streams:',style={'description_width': '120px'})
initial_range_val = 8

#Limit to range variation
threshold = 0.001

#Global
results_min_df = []


#Define range of fan speeds and wet bulbs temp
fan_speeds = np.arange(100, 50-2, -0.25)
wet_bulbs_temp = np.arange(5, 41, 1)


def compute_min_result(wet_bulb_temp, fan_speeds, water_flow, n_stream, n_fan, initial_range_val):
  results = []

  for fan_speed in fan_speeds:
    new_input = np.array([[water_flow, fan_speed, initial_range_val, wet_bulb_temp]])
    new_input_poly = poly.transform(new_input)
    predicted_y = model_poly.predict(new_input_poly)[0]

    previous_range = initial_range_val

    while True:

      effectivenes, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption = new_reange_calculation(predicted_y, n_stream, n_fan, fan_speed, water_flow,initial_range_val )

      if abs(new_range - previous_range) < threshold:
        break

      previous_range = new_range
      new_input = np.array([[water_flow, fan_speed, previous_range, wet_bulb_temp]])
      new_input_poly = poly.transform(new_input)
      predicted_y = model_poly.predict(new_input_poly)[0]

    results.append([fan_speed,new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption, predicted_y])

  min_result = min(results, key=lambda x: x[2])
  return min_result



# Function to update the plot
def update_plot(water_flow, n_stream):

    global results_min_df
    '''
    if n_stream == 4:
        water_flow = 100
        print(f"when n_stream is 4, water flow is always 100%")
    '''

    # Fan number calculation
    n_fan = calculate_fan_number(n_stream)

    results = Parallel(n_jobs=-1)(delayed(compute_min_result)(wet_bulb_temp, fan_speeds, water_flow, n_stream, n_fan, initial_range_val) for wet_bulb_temp in wet_bulbs_temp
                       )
    # Store the fan speeds for each wet bulb temperature and n_stream
    min_fan_speeds = [result[0] for result in results]
    min_consumptions = [result[2] for result in results]


    # Plot the results
    plt.figure(figsize=(12, 6))
    plt.plot(wet_bulbs_temp, min_fan_speeds, label=f'n_stream = {n_stream}')
    plt.xlabel('Inlet Wet Bulb Temperature (°C)')
    plt.ylabel('Fan Speed (%)')
    plt.title(f'Inlet wet bulb temperature vs fan speed when total consumption is minimum with n_stream = {n_stream} with water flow = {water_flow}%')
    plt.legend()
    plt.grid(True)
    plt.show()




    # medir tiempo
    end_time = time.time()

    total_time = end_time - start_time
    print(f"time: {total_time:.4f} seconds")

# Display the interactive plot
interact(update_plot, water_flow=water_flow_slider, n_stream=n_stream_slider);


interactive(children=(FloatSlider(value=67.0, description='Water Flow(%):', min=50.0, step=1.0, style=SliderSt…

# 3-1. Cold Water Temperature vs Wet Bulb Temperature

Using the user-defined inputs (water flow, number of streams, initial range value (default set to 8)), the user can generate a 2D plot showing the cold water temperature values at different wet bulb temperature values when consumption is minimized.  

The goal of this section is to identify the cold water temperature points corresponding to minimum consumption across various wet bulb temperature levels.

In [ ]:
#3-1. Plot Inlet Wet bulb temperature vs Cold water temperature in differnet number of streams + PUMP power APPLIED

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import time

#ejecucin paralela
from joblib import Parallel, delayed

# Medir tiempo

start_time = time.time()

warnings.filterwarnings('ignore')

#Open model
with open('polynomial_regression_model.pkl', 'rb') as file:
    model_poly = pickle.load(file)

#Open polynomial features
with open('polynomial_features.pkl', 'rb') as file:
    poly = pickle.load(file)


# Variable slidar configuration
water_flow_slider = widgets.FloatSlider(value=67, min=50, max=100, step=1, description='Water Flow(%):',style={'description_width': '100px'})
n_stream_slider = widgets.FloatSlider(value=4, min=1, max=4, step=1, description='Number of Streams:',style={'description_width': '120px'})
initial_range_val = 8

#Limit to range variation
threshold = 0.001

#Global
results_min_df = []


#Define range of fan speeds and wet bulbs temp
fan_speeds = np.arange(100, 50-2, -0.25)
wet_bulbs_temp = np.arange(5, 41, 1)



def compute_min_result(wet_bulb_temp, fan_speeds, water_flow, n_stream, n_fan, initial_range_val):
  results = []

  for fan_speed in fan_speeds:
    new_input = np.array([[water_flow, fan_speed, initial_range_val, wet_bulb_temp]])
    new_input_poly = poly.transform(new_input)
    predicted_y = model_poly.predict(new_input_poly)[0]

    previous_range = initial_range_val

    while True:

      effectivenes, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption = new_reange_calculation(predicted_y, n_stream, n_fan, fan_speed, water_flow,initial_range_val )

      if abs(new_range - previous_range) < threshold:
        break

      previous_range = new_range
      new_input = np.array([[water_flow, fan_speed, previous_range, wet_bulb_temp]])
      new_input_poly = poly.transform(new_input)
      predicted_y = model_poly.predict(new_input_poly)[0]

    results.append([fan_speed,new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption, predicted_y])

  min_result = min(results, key=lambda x: x[2])
  return min_result



# Function to update the plot
def update_plot(water_flow, n_stream):

    global results_min_df

    '''
    if n_stream == 4:
        water_flow = 100
        print(f"when n_stream is 4, water flow is always 100%")
    '''

    # Fan number calculation
    n_fan = calculate_fan_number(n_stream)

    results = Parallel(n_jobs=-1)(delayed(compute_min_result)(wet_bulb_temp, fan_speeds, water_flow, n_stream, n_fan, initial_range_val) for wet_bulb_temp in wet_bulbs_temp
                       )
    # Store the fan speeds for each wet bulb temperature and n_stream
    min_fan_speeds = [result[0] for result in results]
    #min_consumptions = [result[2] for result in results]
    min_cold_waters = [result[6] for result in results]

    # Plot the results
    plt.figure(figsize=(12, 6))
    plt.plot(wet_bulbs_temp, min_cold_waters, label=f'n_stream = {n_stream}')
    plt.xlabel('Inlet Wet Bulb Temperature (°C)')
    plt.ylabel('cold water temperature (°C)')
    plt.title(f'Inlet wet bulb temperature vs cold water temperatures when total consumption is minimum when n_stream = {n_stream} with water flow = {water_flow}%')
    plt.legend()
    plt.grid(True)
    plt.show()

    # medir tiempo
    end_time = time.time()

    total_time = end_time - start_time
    print(f"time: {total_time:.4f} seconds")

# Display the interactive plot
interact(update_plot, water_flow=water_flow_slider, n_stream=n_stream_slider);


interactive(children=(FloatSlider(value=67.0, description='Water Flow(%):', min=50.0, step=1.0, style=SliderSt…

# 4. Minimum Total Consumption vs Wet Bulb Temperature

Using the user-defined inputs (water flow, number of streams, initial range value (default set to 8)), the user can generate a 2D plot showing the values of minimum total consumption at different wet bulb temperature levels when consumption is minimized.  

The goal of this section is to observe the minimum total consumption points across various wet bulb temperature levels.

In [ ]:
#4. Plot Inlet Wet bulb temperature vs minimum total consumption in differnet number of streams + PUMP power APPLIED

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import time

# Medir tiempo

start_time = time.time()

# El modelo se encuentra en:a

warnings.filterwarnings('ignore')

#Open model
with open('polynomial_regression_model.pkl', 'rb') as file:
    model_poly = pickle.load(file)

#Open polynomial features
with open('polynomial_features.pkl', 'rb') as file:
    poly = pickle.load(file)

# 슬라이더 설정
water_flow_slider = widgets.FloatSlider(value=67, min=50, max=100, step=1, description='Water Flow(%):',style={'description_width': '100px'})
n_stream_slider = widgets.FloatSlider(value=4, min=1, max=4, step=1, description='Number of Streams:',style={'description_width': '120px'})
initial_range_val = 8

#Limit to range variation
threshold = 0.001

#Global
results_min_df = []


# Function to update the plot
def update_plot(water_flow, n_stream):

    global results_min_df
    results = []
    #fan_speeds = np.arange(100, minimum_fan_speed_percentage - 2, -0.25)
    fan_speeds = np.arange(100, 50-2, -0.25)
    wet_bulbs_temp = np.arange(5, 41, 1)

    '''
    if n_stream == 4:
        water_flow = 100
        print(f"when n_stream is 4, water flow is always 100%")
    '''

    # Fan number calculation
    n_fan = calculate_fan_number(n_stream)

    # Store the fan speeds for each wet bulb temperature and n_stream
    min_fan_speeds = []
    min_consumptions = []

    for x_val in wet_bulbs_temp:
        results = []

        for fan_speed in fan_speeds:
            # Cold water temperature prediction
            new_input = np.array([[water_flow, fan_speed, initial_range_val, x_val]])
            new_input_poly = poly.transform(new_input)
            predicted_y = model_poly.predict(new_input_poly)[0]


            # Calculations
            effectivenes, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption = new_reange_calculation(predicted_y, n_stream, n_fan, fan_speed, water_flow,initial_range_val )

            previous_range = initial_range_val

            # Iterative loop
            while abs(new_range - previous_range) > threshold:
                previous_range = new_range

                # Cold water temperature prediction
                new_input = np.array([[water_flow, fan_speed, previous_range, x_val]])
                new_input_poly = poly.transform(new_input)
                predicted_y = model_poly.predict(new_input_poly)[0]

                # Calculations

                effectivenes, new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption = new_reange_calculation(predicted_y, n_stream, n_fan, fan_speed, water_flow,initial_range_val )

            #Append when the total consumption for one fan speed
            results.append([fan_speed,new_range, total_consumption, fan_consumption, chiller_consumption, pump_consumption, predicted_y])

        # Find fan speed con minimum total consumption in x wet bulb temperature
        min_result = min(results, key=lambda x: x[2])  # fila con total consumption minima
        min_fan_speeds.append(min_result[0])
        min_consumptions.append(min_result[2])


    # Plot the results
    plt.figure(figsize=(12, 6))
    plt.plot(wet_bulbs_temp, min_consumptions, label=f'n_stream = {n_stream}')
    plt.xlabel('Inlet Wet Bulb Temperature (°C)')
    plt.ylabel('minimum consupmtion (kWe)')
    plt.title(f'Inlet wet bulb temperature vs minimum total consumption with n_stream = {n_stream} with water flow = {water_flow}%')
    plt.legend()
    plt.grid(True)
    plt.show()

    # medir tiempo
    end_time = time.time()

    total_time = end_time - start_time
    print(f"time: {total_time:.4f} seconds")

# Display the interactive plot
interact(update_plot, water_flow=water_flow_slider, n_stream=n_stream_slider);




interactive(children=(FloatSlider(value=67.0, description='Water Flow(%):', min=50.0, step=1.0, style=SliderSt…